In [0]:
%tensorflow_version 1.x

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import re
from collections import Counter
import gc

In [22]:
tf.__version__

'1.15.2'

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
urlOriginalSentence='/content/drive/My Drive/Colab Notebooks/Work/Dataset/original'
urlTargetSentence='/content/drive/My Drive/Colab Notebooks/Work/Dataset/target'

In [0]:
def loadDataset(url,intInd,lastInd):
    sentences=[]
    with open(url,'r') as f:
        sentences=f.read().split('\n')[intInd:lastInd]
    return sentences

In [0]:
startInd=0
lastInd=285006
originalSentence=loadDataset(urlOriginalSentence,startInd,lastInd)
targetSentence=loadDataset(urlTargetSentence,startInd,lastInd)

originalEvalSentence=loadDataset(urlOriginalSentence,lastInd,lastInd+1000)
targetEvalSentence=loadDataset(urlTargetSentence,lastInd,lastInd+1000)

In [0]:
originalSentence=originalSentence[6:]
targetSentence=targetSentence[6:]

In [28]:
max_length_original=20 
max_length_target=20
print(max_length_original)
print(max_length_target)
vocabulary_size=15000

20
20


In [0]:
def cleanOriginalSentence(sentences,wordList=None, addHeaders=False,max_length=None):
    cleanedSentences=[]
    for sentence in sentences:
        sentence=re.sub('[.,\'\"?~!#@$%^&*()-]+',"",sentence.lower())
        sentence=re.sub("[ ]+"," ",sentence)
        sentence=re.sub('[^a-zA-Z]+'," ",sentence)
        if len(sentence.split(" "))>max_length-2:
            sentence=" ".join(sentence.split(" ")[:max_length-2])
        if wordList is not None:
            temp=[]
            for word in sentence.split(" "):
                if word in wordList:
                    temp.append(word)
                else:
                    temp.append('unk')
            if len(temp)>max_length-2:
                temp=temp[:max_length-2]
            sentence=" ".join(temp)
        sentence=' '.join([w for w in sentence.split(' ')])
        if addHeaders==True:
            sentence='<start> ' + sentence+ ' <end>'

        cleanedSentences.append(sentence)
    return cleanedSentences

In [0]:
def cleanTargetSentence(sentences,wordList=None,addHeaders=False,max_length=None):
    cleanedSentences=[]
    for sentence in sentences:
        sentence=re.sub('\w-\w*'," ", sentence.lower())
        sentence=re.sub('[.,\'\"?~!#@$%^&*()-]+',"",sentence)
        sentence=re.sub("\s+"," ",sentence)
        sentence=re.sub('[^а-яА-Я]+'," ",sentence) 
        
        if len(sentence.split(" "))>max_length-2:
            sentence=" ".join(sentence.split(" ")[:max_length-2])    
            sentence=sentence.strip()    
            
        if wordList is not None:
            temp=[]
            for word in sentence.split(" "):
                if word in wordList:
                    temp.append(word)
                else:
                    temp.append('unk')
            if len(temp)>max_length-2:
                temp=temp[:max_length-2]
            sentence=" ".join(temp)
            
        sentence=' '.join([w for w in sentence.split(" ")])
        if addHeaders==True:
            sentence='<start> ' + sentence.strip()+ ' <end>'

        cleanedSentences.append(sentence)
    return cleanedSentences

In [31]:
originalSentence=cleanOriginalSentence(originalSentence,max_length=max_length_original)
counterOriginal=Counter(" ".join(originalSentence).split(" ")).most_common(vocabulary_size)
counterOriginal={word:k for word,k in counterOriginal }
print(len(counterOriginal))

targetSentence=cleanTargetSentence(targetSentence,max_length=max_length_target)
counterTarget=Counter(" ".join(targetSentence).split(" ")).most_common(vocabulary_size)
counterTarget={word:k for word,k in counterTarget }
print(len(counterTarget))

15000
15000


In [0]:
originalSentence=cleanOriginalSentence(originalSentence,counterOriginal.keys(),True,max_length_original)
originalEvalSentence=cleanOriginalSentence(originalEvalSentence,counterOriginal.keys(),True,max_length_original)

targetSentence=cleanTargetSentence(targetSentence,counterTarget.keys(),True,max_length_target)
targetEvalSentence=cleanTargetSentence(targetEvalSentence,counterTarget.keys(),True,max_length_target)

In [33]:
targetEvalSentence

['<start> тунис <end>',
 '<start> турция <end>',
 '<start> соединенные штаты америки <end>',
 '<start> уругвай <end>',
 '<start> венесуэла <end>',
 '<start> вьетнам <end>',
 '<start> югославия <end>',
 '<start> заир <end>',
 '<start> замбия <end>',
 '<start> зимбабве <end>',
 '<start>  <end>',
 '<start>  <end>',
 '<start> доклад генерального секретаря в связи со всеобъемлющим пересмотром <end>',
 '<start> добавление <end>',
 '<start> в своем докладе от ноября года п я рекомендовал совету продлить мандат вооруженных сил организации объединенных наций по <end>',
 '<start> хочу сообщить совету что правительство кипра а также правительства греции и соединенного королевства великобритании и северной ирландии указали <end>',
 '<start> правительство турции указало что разделяет и поддерживает позицию кипрск стороны unk на предыдущих заседаниях совета безопасности по поводу <end>',
 '<start>  <end>',
 '<start>  <end>',
 '<start>  <end>',
 '<start> письмо генерального секретаря от ноября года <

In [0]:
def tokenize(sentences):
    tokenizer=keras.preprocessing.text.Tokenizer(filters="",)
    tokenizer.fit_on_texts(sentences)
    tensor=tokenizer.texts_to_sequences(sentences)
    return np.array(tensor),tokenizer

In [0]:
original,originalTokenizer=tokenize(originalSentence)
target,targetTokenizer=tokenize(targetSentence)


In [0]:

originalEval=np.array(originalTokenizer.texts_to_sequences(originalEvalSentence))

targetEval=np.array(targetTokenizer.texts_to_sequences(targetEvalSentence))

In [0]:
batch_size=1000
steps_per_epoch=len(original)//batch_size
embedding_dims=300
units=500
vocab_original_size=len(originalTokenizer.index_word)+1
vocab_target_size=len(targetTokenizer.index_word)+1
max_epochs=15

In [0]:
def generator(encoderInp,decoderInp,max_length,batch_size,vocab_size):
    batch_index=0
    hidden=np.array([True  for i in range(batch_size)],dtype='bool').reshape(-1)
    X1,X2,y=[],[],[]
    dummy_range=np.arange(0,len(decoderInp),dtype='int32')
    while 1:
      index=np.random.choice(dummy_range,batch_size)
      batch_max_length=max(len(sentence) for sentence in decoderInp[index])
      target_sequence=keras.preprocessing.sequence.pad_sequences(decoderInp[index],maxlen=batch_max_length)
      for i in range(1,batch_max_length-1):
          X1=keras.preprocessing.sequence.pad_sequences(encoderInp[index],maxlen=max_length,padding='pre')
          X2=keras.preprocessing.sequence.pad_sequences(target_sequence[:,0:i],maxlen=max_length-1,padding='pre')
          y=keras.utils.to_categorical(target_sequence[:,i],num_classes=vocab_size)
          yield ([np.array(X1),np.array(X2),hidden],np.array(y))
          X1,X2,y=[],[],[]
          hidden=np.array([False for i in range(batch_size)],dtype='bool' ).reshape(-1)
      batch_index=(batch_index+batch_size)%(len(encoderInp))
      hidden=np.array([True for i in range(batch_size)],dtype='bool' ).reshape(-1)
                      

In [0]:
train_generator=generator(original,target,max_length_target,batch_size,vocab_target_size)
val_generator=generator(originalEval,targetEval,max_length_target,batch_size//2,vocab_target_size)

In [0]:
class Encoder(keras.Model):
    def __init__(self,units,batch_size,vocabulary_size,embedding_dims):
        super(Encoder,self).__init__()
        self.batch_size=batch_size
        self.units=units
        self.encoder_embedding=keras.layers.Embedding(vocabulary_size,embedding_dims,name='encoder_embedding')
        self.rnn_layer=keras.layers.GRU(units,return_sequences=True,return_state=True,name='encoder_rnn')
    def call(self,inputX):
        embed_out=self.encoder_embedding(inputX)
        rnn_out,rnn_hidden=self.rnn_layer(embed_out)
        return rnn_out,rnn_hidden

In [0]:
class AttentionBahdanau(keras.layers.Layer):
    def __init__(self,units):
        super(AttentionBahdanau,self).__init__()
        self.dense1=keras.layers.Dense(units)
        self.dense2=keras.layers.Dense(units)
        self.dense3=keras.layers.Dense(1,activation='softmax')
    def call(self,inputs):
        annotations,hidden=inputs
        #print('Annotations: ',annotations)
        hidden=keras.backend.expand_dims(hidden,1)
        #print("Expanded hidden: ",hidden)
        energy=keras.activations.tanh(self.dense1(annotations)+self.dense2(hidden))
        #print("Energy: ",energy)
        attention_weights=self.dense3(energy)
        #print("Attention_weights: ",attention_weights)
        context_vector=attention_weights*annotations
        #print("Context_matrix: ",context_vector)
        context_vector=keras.backend.sum(context_vector,axis=1)
        #print("Context_vector: ",context_vector)
        return context_vector,attention_weights

In [0]:
class Decoder(keras.Model):
    def __init__(self,units,batch_size,vocabulary_size,embedding_dims):
        super(Decoder,self).__init__()
        self.batch_size=batch_size
        self.units=units
        self.decoder_embedding=keras.layers.Embedding(vocabulary_size,embedding_dims,name='decoder_embedding')
        self.rnn_layer=keras.layers.GRU(units,return_state=True,name='decoder_rnn')
        self.attention=AttentionBahdanau(units)
        self.last_layer=keras.layers.Dense(vocabulary_size,activation='softmax',name='Last_layer')
        self.hidden=[]
    def call(self,inputs):
        annotation,decoder_input,self.hidden,enc_hidder=inputs
        context,attention_weights=self.attention([annotation,self.hidden])
        embed_out=self.decoder_embedding(decoder_input)
        #print("Embeding_out: ",embed_out)
        context=keras.layers.RepeatVector(embed_out.shape[1])(context)
        #print("Expanded context: ",context)
        concated = tf.concat([embed_out, context], axis=-1)
        #print("Concated: ",concated)
        out,self.hidden=self.rnn_layer(concated,initial_state=[enc_hidder])
        #print("Out: ",out)
        #print("New Hidden: ",self.hidden)
        out=self.last_layer(out)
        #print("Output: ",out)
        return out,self.hidden,attention_weights
        
        

In [0]:
class AttentionModel(keras.Model):
  def __init__(self,units,batch_size,vocabulary_size_encoder,vocabulary_size_target,embedding_dims):
    super(AttentionModel,self).__init__()
    self.batch_size=batch_size
    self.units=units
    self.encoder=Encoder(units,batch_size,vocabulary_size_encoder,embedding_dims)
    self.decoder=Decoder(self.units,batch_size,vocabulary_size_target,embedding_dims)
    self.hidden=None
    self.weight=None
  def call(self,inputs):
    enc_input,decoder_input,decoder_hidden=inputs
    annotations,enc_hidden=self.encoder(enc_input)
    print(self.hidden)
    print(enc_hidden)
    if self.hidden is None:
      self.hidden=enc_hidden
    else:
      self.hidden=tf.where(decoder_hidden,enc_hidden,self.hidden)
    print(decoder_hidden)    
    out,self.hidden,weights=self.decoder([annotations,decoder_input,self.hidden,enc_hidden])
    self.weight=weights
    return out


In [0]:
encoder_input=keras.Input((max_length_original,),dtype='float32')
decoder_input=keras.Input((max_length_target-1,),dtype='float32')
decoder_hidden=keras.Input(shape=(),dtype='bool')

In [45]:
attentionModel=AttentionModel(units,batch_size,vocab_original_size,vocab_target_size,embedding_dims)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [46]:
output=attentionModel([encoder_input,decoder_input,decoder_hidden])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
None
Tensor("attention_model/encoder/encoder_rnn/while/Exit_2:0", shape=(?, 500), dtype=float32)
Tensor("input_3:0", shape=(?,), dtype=bool)


In [0]:
model=keras.Model([encoder_input,decoder_input,decoder_hidden],[output])

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/Work/ImageCaptioning/Image Caption With Attention/glove.6B.300d.txt') as f:
    lines=f.read()

embedding_index={}
for line in lines.split('\n'):
    values=line.split(" ")
    word=values[0]
    values=np.asarray(values[1:],dtype='float64')
    if word in originalTokenizer.word_index.keys():
        embedding_index[word]=values

embedding_matrix=np.zeros((vocab_original_size,embedding_dims),dtype='float64')

for word,i in originalTokenizer.word_index.items():
    vec=embedding_index.get(word,None)
    if vec is not None:
        embedding_matrix[i]=vec
        
        


In [0]:
attentionModel.encoder.encoder_embedding.set_weights([embedding_matrix])
attentionModel.encoder.encoder_embedding.trainable=False

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/Work/Machine Translation/NMT_Attention/NMT_Badhanau.hdf5')

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [52]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 19)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
attention_model (AttentionModel (None, 15003)        20172804    input_1[0][0]                    
                                                                 input_2[0][0]                

In [0]:
del(embedding_matrix)
del(embedding_index)
del(lines)

In [0]:
callback=keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/Work/Machine Translation/NMT_Attention/NMT_Badhanau.hdf5',save_best_only=True,save_weights_only=True,monitor='val_loss')

In [38]:
model.fit(train_generator,steps_per_epoch=len(target)//batch_size*(max_length_target-1),epochs=10,callbacks=[callback],validation_data=(val_generator),validation_steps=len(targetEval)//batch_size*(max_length_target-1))

Epoch 1/10
5415/5415 [==============================] - 1286s 237ms/step - loss: 3.2486 - acc: 0.4659 - val_loss: 3.1910 - val_acc: 0.4619
Epoch 2/10
5415/5415 [==============================] - 1308s 242ms/step - loss: 2.3538 - acc: 0.5510 - val_loss: 2.9763 - val_acc: 0.4829
Epoch 3/10
5415/5415 [==============================] - 1294s 239ms/step - loss: 2.1466 - acc: 0.5754 - val_loss: 3.0931 - val_acc: 0.4752
Epoch 4/10
5415/5415 [==============================] - 1289s 238ms/step - loss: 2.0400 - acc: 0.5879 - val_loss: 3.2476 - val_acc: 0.4582
Epoch 5/10
5415/5415 [==============================] - 1292s 239ms/step - loss: 1.9689 - acc: 0.5959 - val_loss: 3.0205 - val_acc: 0.4885
Epoch 6/10
5415/5415 [==============================] - 1293s 239ms/step - loss: 1.9176 - acc: 0.6027 - val_loss: 3.0034 - val_acc: 0.4994
Epoch 7/10
5415/5415 [==============================] - 1298s 240ms/step - loss: 1.8778 - acc: 0.6076 - val_loss: 3.2909 - val_acc: 0.4709
Epoch 8/10
5415/5415 [=====

In [69]:
print("Done")

Done


In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/Work/Machine Translation/NMT_Attention/NMT_Badhanau.hdf5')

In [90]:
encoder=Encoder(units,batch_size,vocab_original_size,embedding_dims)
decoder=Decoder(units,batch_size,vocab_target_size,embedding_dims)

decoder_hidden=keras.Input((units,),dtype='float32')
encoder_hidden=keras.Input((units,),dtype='float32')
encoder_annotaion=keras.Input((max_length_original,units),dtype='float32')

enc_out,enc_hidden=encoder(encoder_input)
dec_out,dec_hidden,dec_weight=decoder([encoder_annotaion,decoder_input,decoder_hidden,encoder_hidden])

encoder.set_weights(attentionModel.encoder.get_weights())
decoder.set_weights(attentionModel.decoder.get_weights())



In [0]:
targetTokenizer.index_word[0]='unk'

In [114]:
ind=555
inp1=keras.preprocessing.sequence.pad_sequences([np.array((original[ind]),dtype='float32')],maxlen=max_length_original)
inp2='<start>'
pred=[targetTokenizer.word_index[inp2]]
text=""
y_hat=keras.preprocessing.sequence.pad_sequences([pred],maxlen=max_length_target-1,padding='pre')
hidder=np.array(True).reshape(-1)

for i in range(len(originalEval[ind])-2):
    #y_hat=decoder.predict([inp1,seq,hidden])
    y_hat=model.predict([inp1,y_hat,hidder])
    y_hat=np.argmax(y_hat[0])
    hidder=np.array(False).reshape(-1)
    pred.append(y_hat)
    text=text+ " " + targetTokenizer.index_word[y_hat]
    if targetTokenizer.index_word[y_hat]=='<end>':
        break
    y_hat=keras.preprocessing.sequence.pad_sequences([pred],maxlen=max_length_target-1,padding='pre')

print(text)

 обслуживания общее общее ф п п п unk п п п unk п п п п п unk


In [112]:
originalEvalSentence[ind]

'<start> p p p general service <end>'

In [113]:

targetSentence[ind]

'<start> общего обслуживания <end>'

In [0]:
targetEval

array([[   1,   21, 2756, ...,    0,    0,    0],
       [   1, 1278,    8, ...,    0,    0,    0],
       [   1,  490,  139, ...,  189,    2,    0],
       ...,
       [   1,   26, 1900, ...,    0,    0,    0],
       [   1,   20,   22, ...,    0,    0,    0],
       [   1,   47,   55, ...,    0,    0,    0]], dtype=int32)

In [0]:
original[ind][::-1]

array([  3,  85, 834,   5, 246, 100,   8, 425,   2,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [0]:
from nltk.translate.bleu_score import corpus_bleu
def eval(originalEval,targetEval):
  actual,predicted=[],[]
  for sentence,target in zip(originalEval,targetEval):
    inp1=keras.preprocessing.sequence.pad_sequences([np.array((sentence),dtype='float32')],maxlen=max_length_original)
    inp2='<start>'
    pred=[targetTokenizer.word_index[inp2]]
    y_hat=keras.preprocessing.sequence.pad_sequences([pred],maxlen=max_length_target-1,padding='pre')
    hidder=np.array(True).reshape(-1)
    text=""
    for i in range(len(sentence)-2):
        y_hat=model.predict([inp1,y_hat,hidder])
        y_hat=np.argmax(y_hat[0])
        hidder=np.array(False).reshape(-1)
        pred.append(y_hat)
        text=text+ " " + targetTokenizer.index_word[y_hat]
        if targetTokenizer.index_word[y_hat]=='<end>':
            break
        y_hat=keras.preprocessing.sequence.pad_sequences([pred],maxlen=max_length_target-1,padding='pre')
    predicted.append(' '.join(text.split(' ')[:-1]))
    actual.append(target)
    print(len(actual)/len(targetEval))
  return actual,predicted


In [117]:
actual,predicted=eval(originalEval,targetEvalSentence)

0.001
0.002
0.003
0.004
0.005
0.006
0.007
0.008
0.009
0.01
0.011
0.012
0.013
0.014
0.015
0.016
0.017
0.018
0.019
0.02
0.021
0.022
0.023
0.024
0.025
0.026
0.027
0.028
0.029
0.03
0.031
0.032
0.033
0.034
0.035
0.036
0.037
0.038
0.039
0.04
0.041
0.042
0.043
0.044
0.045
0.046
0.047
0.048
0.049
0.05
0.051
0.052
0.053
0.054
0.055
0.056
0.057
0.058
0.059
0.06
0.061
0.062
0.063
0.064
0.065
0.066
0.067
0.068
0.069
0.07
0.071
0.072
0.073
0.074
0.075
0.076
0.077
0.078
0.079
0.08
0.081
0.082
0.083
0.084
0.085
0.086
0.087
0.088
0.089
0.09
0.091
0.092
0.093
0.094
0.095
0.096
0.097
0.098
0.099
0.1
0.101
0.102
0.103
0.104
0.105
0.106
0.107
0.108
0.109
0.11
0.111
0.112
0.113
0.114
0.115
0.116
0.117
0.118
0.119
0.12
0.121
0.122
0.123
0.124
0.125
0.126
0.127
0.128
0.129
0.13
0.131
0.132
0.133
0.134
0.135
0.136
0.137
0.138
0.139
0.14
0.141
0.142
0.143
0.144
0.145
0.146
0.147
0.148
0.149
0.15
0.151
0.152
0.153
0.154
0.155
0.156
0.157
0.158
0.159
0.16
0.161
0.162
0.163
0.164
0.165
0.166
0.167
0.168
0.169
0.1

In [118]:
  print("BLEU-1: %f" % corpus_bleu(actual,predicted,weights=(1,0,0,0)))
  print("BLEU-2: %f" % corpus_bleu(actual,predicted,weights=(0.5,0.5,0,0)))
  print("BLEU-3: %f" % corpus_bleu(actual,predicted,weights=(0.33,0.33,0.33,0)))
  print("BLEU-4: %f" % corpus_bleu(actual,predicted,weights=(0.25,0.25,0.25,0.25)))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.216648
BLEU-2: 0.465455
BLEU-3: 0.603670
BLEU-4: 0.682243
